## Emotion_Datasets 

In [ ]:
# Imports 

import numpy as np 
import pandas as pd 
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import os
import glob
import json

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout, RandomFlip
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.data import Dataset
from tensorflow.keras.losses import SparseCategoricalCrossentropy


### VREED

In [ ]:
#Import VREED Dataset

#Questionnaires 
VREEDpreQ = pd.read_excel('/Users/emilydoherty/Desktop/IRES/Emotion_Datasets/VREED/Self-Reported Questionnaires/Participant Profile  Pre-Exposure Ratings.xlsx')
VREEDpostQ=pd.read_excel('/Users/emilydoherty/Desktop/IRES/Emotion_Datasets/VREED/Self-Reported Questionnaires/Post Exposure Ratings.xlsx')

#Features (min, max, etc.)
VREEDeye=pd.read_csv('/Users/emilydoherty/Desktop/IRES/Emotion_Datasets/VREED/Eye Tracking Data/Eye Tracking Data (Features Extracted)/EyeTracking_FeaturesExtracted.csv')
VREEDecg=pd.read_csv('/Users/emilydoherty/Desktop/IRES/Emotion_Datasets/VREED/ECG-GSR Data/ECG-GSR (Features Extracted)/ECG_FeaturesExtracted.csv')
VREEDgsr=pd.read_csv('/Users/emilydoherty/Desktop/IRES/Emotion_Datasets/VREED/ECG-GSR Data/ECG-GSR (Features Extracted)/GSR_FeaturesExtracted.csv')


In [ ]:
#dat file

### CEAP

In [ ]:
#Import CEAP Data
#Refer to ceap_example_notebook

# Post-processed .csv
CEAPpost=pd.read_csv('/Users/emilydoherty/Desktop/Data Jupyter Notebook_NEW/temp/ceap_example/Dataset_CEAP_postprocessed.csv')

#Load CEAP Features .csv


### AVReality 
this dataset will need to be filtered - hopefully use same filtering techniques as CEAP to extract features once that script is debugged

In [ ]:
# AV Reality Data
AVRq=pd.read_excel('/Users/emilydoherty/Desktop/IRES/Emotion_Datasets/AVReality/Data_new.xlsx',sheet_name="Sheet1")
AVRq.describe()
AVRq.head()

In [ ]:
#Nature BVP Files 
path = '/Users/emilydoherty/Desktop/IRES/Emotion_Datasets/AVReality/E4/Nature/BVP'
csv_files = glob.glob(path + "/*.csv")
csv_files=sorted(csv_files)

# Read each CSV file into DataFrame
# This creates a list of dataframes
N_BVP=[]
for file in csv_files:
	df_list = (pd.read_csv(file))
	df_list['filename']= os.path.basename(file)
	df_list['cond']= 'Nature'
	N_BVP.append(df_list)

N_BVP = pd.concat(N_BVP)

N_BVP=N_BVP.drop(columns=['Event Id','Event Date','Event Duration'], axis=1)

In [ ]:
#Subway BVP Files 
path = '/Users/emilydoherty/Desktop/IRES/Emotion_Datasets/AVReality/E4/Subway/BVP'
csv_files = glob.glob(path + "/*.csv")
csv_files=sorted(csv_files)

# Read each CSV file into DataFrame
# This creates a list of dataframes
S_BVP=[]
for file in csv_files:
	df_list = (pd.read_csv(file))
	df_list['filename']= os.path.basename(file)
	df_list['cond']= 'Subway'
	S_BVP.append(df_list)

S_BVP = pd.concat(S_BVP)

S_BVP=S_BVP.drop(columns=['Event Id','Event Date','Event Duration'], axis=1)

In [ ]:
#All Blood Vol Pulse data
All_BVP = [N_BVP, S_BVP]
All_BVP= pd.concat(All_BVP)

#Now apply filtering 


In [ ]:
#Nature GSR Files 
path = '/Users/emilydoherty/Desktop/IRES/Emotion_Datasets/AVReality/E4/Nature/GSR'
csv_files = glob.glob(path + "/*.csv")
csv_files=sorted(csv_files)

# Read each CSV file into DataFrame
# This creates a list of dataframes
N_GSR=[]
for file in csv_files:
	df_list = (pd.read_csv(file))
	df_list['filename']= os.path.basename(file)
	df_list['cond']= 'Nature'
	N_GSR.append(df_list)

N_GSR = pd.concat(N_GSR)

N_GSR=N_GSR.drop(columns=['Event Id','Event Date','Event Duration'], axis=1)

In [ ]:
#Subway GSR Files 
path = '/Users/emilydoherty/Desktop/IRES/Emotion_Datasets/AVReality/E4/Subway/GSR'
csv_files = glob.glob(path + "/*.csv")
csv_files=sorted(csv_files)

# Read each CSV file into DataFrame
# This creates a list of dataframes
S_GSR=[]
for file in csv_files:
	df_list = (pd.read_csv(file))
	df_list['filename']= os.path.basename(file)
	df_list['cond']= 'Subway'
	S_GSR.append(df_list)

S_GSR = pd.concat(S_GSR)

S_GSR=S_GSR.drop(columns=['Event Id','Event Date','Event Duration'], axis=1)

In [ ]:
#All GSR data
All_GSR = [N_GSR, S_GSR]
All_GSR= pd.concat(All_GSR)

#Now apply filtering 

In [ ]:
#Nature TMP Files 
path = '/Users/emilydoherty/Desktop/IRES/Emotion_Datasets/AVReality/E4/Nature/TMP'
csv_files = glob.glob(path + "/*.csv")
csv_files=sorted(csv_files)

# Read each CSV file into DataFrame
# This creates a list of dataframes
N_TMP=[]
for file in csv_files:
	df_list = (pd.read_csv(file))
	df_list['filename']= os.path.basename(file)
	df_list['cond']= 'Nature'
	N_TMP.append(df_list)

N_TMP = pd.concat(N_TMP)

N_TMP=N_TMP.drop(columns=['Event Id','Event Date','Event Duration'], axis=1)

In [ ]:
#Subway TMP Files 
path = '/Users/emilydoherty/Desktop/IRES/Emotion_Datasets/AVReality/E4/Subway/TMP'
csv_files = glob.glob(path + "/*.csv")
csv_files=sorted(csv_files)

# Read each CSV file into DataFra
# This creates a list of dataframes
S_TMP=[]
for file in csv_files:
	df_list = (pd.read_csv(file))
	df_list['filename']= os.path.basename(file)
	df_list['cond']= 'Subway'
	S_TMP.append(df_list)

S_TMP = pd.concat(S_TMP)

S_TMP=S_TMP.drop(columns=['Event Id','Event Date','Event Duration'], axis=1)

In [ ]:
#All TMP data
All_TMP = [N_TMP, S_TMP]
All_TMP= pd.concat(All_TMP)

#Now apply filtering 

